In [ ]:
# В этом задании вам необходимо будет реализовать статистический Spell Checker

In [ ]:
#!pip install razdel corus numpy nltk tqdm
#!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [1]:
import re
import razdel
import corus
import numpy as np
import nltk
import tqdm
import collections
from collections import Counter

In [ ]:
# Данные: Корпус русских текстов для n-gram статистики --> возьмем новостный корпус с corus
# Словарь слов русского языка (чем больше, тем лучше)
# Предложения, которые необходимо исправить

In [ ]:
# Шаг 1:

# На первом шаге коррекции наших текстов определим такие токены, которым требуется исправление.
# Для этого проведем статистическую бинарную классификацию токенов в наших предложениях
# (1- токен содержит опечатку, 0- токен не содержит опечатку)

# Определять неправильные токены будем с помощью формулы расчета "подозрительности" триграмм из статьи 1975 года 
# "Computer Detection of Typographical Errors  R. Morris, L. Cherry". Статья приложена.

# Сначала напишем формулу для получения n-gram слова. Для формулы нам нужны только биграммы и триграммы, но мы напишем
# функцию, которая возвращает n-граммы для любого заданного n.

In [2]:
# как --> ['как']
# не --> []
# шарик --> ['шар', 'ари', 'рик']
# неправильный --> ['неп', 'епр', 'пра', 'рав', 'ави', 'вил', 'иль', 'льн', 'ьны', 'ный']

def ngram(word, n):
    ngrams=[]
    for i in range(len(word)):
        ngram = "".join(word[i:i+n])
        if  len (ngram) < n:
            break
        ngrams.append(ngram)
    return ngrams
    
assert ngram('неправильный', 3) == [''.join(g) for g in list(nltk.ngrams('неправильный', 3))]

In [3]:
ngram("неправильный",3)

['неп', 'епр', 'пра', 'рав', 'ави', 'вил', 'иль', 'льн', 'ьны', 'ный']

In [ ]:
# Логика сбора статистики такова:
# 1) Идем по текстам корпуса новостей
# 2) Токенизируем тексты с помощью razdel.tokenize()
# 3) Приводим каждый токен к нижнему регистру
# 4) Токены, которые содержат только символы кириллицы, копим в статистику 
#    (делим токен на биграмы и триграммы и копим статистику в Counter)

In [4]:
# Корпус русских текстов
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)


In [5]:
def clean_text(text):
    text= str(text).lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = razdel.tokenize(text)
    text = [t.text for t in text]
    return text

In [6]:
def is_cyrillic(word):
    pattern = re.compile("^[а-яА-Я]+$")
    if pattern.match(word):
        return True
    

In [ ]:
ngram_stats = Counter()

for record in tqdm.tqdm(records):
    #TODO: на основе текстов создать статистику триграм 
    
    tokens = clean_text(record) # Токенизируем текст

    for token in tokens:
        if not is_cyrillic(token):
            continue

        # Нижний регистр
        #token = token.lower()
        
        # Получаем триграммы и биграммы
        n_grams_3 = ngram(token, 3)
        n_grams_2 = ngram(token, 2)
        for g in n_grams_3:
            ngram_stats[g] += 1
        for g2 in n_grams_2:
            ngram_stats[g2] += 1
            
            

In [ ]:
import json

with open("ngrams_dict.json", "w", encoding="utf-8") as file:
    json.dump(ngram_stats, file)

In [7]:
with open('ngrams_dict.json') as json_file:
    ngram_stats = json.load(json_file)

In [8]:
import pandas as pd

df = pd.read_csv('broken_texts.csv.gz', compression='gzip')[['text']]

In [9]:
df['text'].iloc[11]

'в 1996 г . плоучил звание заслуженныйй профессор харьковского государственного университета .'

In [ ]:
#ngram_stats

Формула для расчета подозрительности триграмы (обозначается xyz) выглядит следующим образом:

$$ i(T) = \frac{1}{2}[log(xy) + log(yz)] - log(xyz) $$

Если биграма или триграма отсутствует в словаре, то значение логарифма по задумке авторов сразу равно -10

Эту логику лучше вынести в отдельную функцию

In [10]:
import math

In [11]:
def count_log(g):
    if g in ngram_stats:
        xyz = g
        bigram = ngram(xyz, 2)
        xy, yz = bigram[0], bigram[1]
        log_xy, log_yz, log_xyz = math.log(ngram_stats[xy]), math.log(ngram_stats[yz]), math.log(ngram_stats[xyz])
        return log_xy, log_yz, log_xyz
    else:
        xyz = g
        log_xyz = -10
        bigram = ngram(xyz, 2)
        xy, yz = bigram[0], bigram[1]
        
        if xy in ngram_stats:
            log_xy = math.log(ngram_stats[xy])
        else:
            log_xy = -10

        if yz in ngram_stats:
            log_yz = math.log(ngram_stats[yz])
        else:
            log_yz = -10
        return log_xy, log_yz, log_xyz

In [12]:
# Соберем всё вместе

def pecularity(trigram):
    log_xy, log_yz, log_xyz = count_log(trigram)
    return (0.5 * (log_xy + log_yz) - log_xyz)

def get_scores(token):
#    В конечном итоге скоры для одного слова должны выглядеть как-то так:
#     {'плоучил': 
#          {'пло': 2.59,
#           'лоу': 3.29,
#           'оуч': 4.09,
#           'учи': 1.56,
#           'чил': 2.40}
#     }
    tokens = {}
    trigrams = ngram(token, 3)
    for gr in trigrams:
        tokens[gr] = pecularity(gr)
    return tokens
# Если токен имеет триграмы с скорами > 4, то мы считаем, что такой токен имеет ошибку.
# То есть его частота в нашем корпусе частот практически незначительна
    



In [13]:
get_scores("плоучил")

{'пло': 2.598005001331824,
 'лоу': 3.317309312349309,
 'оуч': 4.06931248280603,
 'учи': 1.5528543967583328,
 'чил': 2.380220017451915}

In [ ]:
# Токены, в которых есть значения выше 4: пробуем восстановить

# По аналогии с решением предыдущей задачи воспользуйтесь n-gram преобразованием, чтобы найти top-k ближайших кандидатов
# для исправления токена с помощью списка слов русского языка и функции scipy.cdist 

# Будем использовать и униграмы, и биграмы, и триграмы для этой части задания


In [14]:
# Слова русского языка
words = list(pd.read_csv('russian_words.zip', compression='zip').values.flatten())

print(words[120:130])

len(words)

['Абакан', 'Абакана', 'Абакане', 'абаканец', 'Абаканом', 'абаканская', 'абаканские', 'абаканский', 'абаканским', 'абаканскими']


1532628

In [16]:
df["processed"] = df['text'].apply(lambda x: x.split())

In [17]:
df["processed"] = df["processed"].apply(lambda x: [w for w in x if is_cyrillic(w) == True])

In [23]:
def check_words_in_df(lst):  # соберем все триграмы для каждого текста
    all_words={}
    for word in lst:
        all_words[word] = get_scores(word)
    return(all_words)
    

In [24]:
df["dict_of_ngrams"] = df["processed"].apply(lambda x: check_words_in_df(x))

In [38]:
df["dict_of_ngrams"][1]

{'интегумент': {'инт': 2.326208970704359,
  'нте': 2.2527588910908705,
  'тег': 3.9023532482785424,
  'егу': 2.8068777168581747,
  'гум': 3.515721293036137,
  'уме': 2.1283376349907766,
  'мен': 1.741324122306505,
  'ент': 1.0945030616112206},
 'от': {},
 'покрывало': {'пок': 2.451738030492022,
  'окр': 2.249554292192645,
  'кры': 1.77561686257339,
  'рыв': 1.9456804895024558,
  'ыва': 1.5859252328129418,
  'вал': 1.854516300002535,
  'ало': 2.0451141925690095},
 'покров': {'пок': 2.451738030492022,
  'окр': 2.249554292192645,
  'кро': 2.945803936295226,
  'ров': 1.5709371735628892},
 'термин': {'тер': 1.720899212346895,
  'ерм': 3.0340463307357055,
  'рми': 2.695759334473074,
  'мин': 1.9837434414451423},
 'лужащий': {'луж': 1.4468967374930557,
  'ужа': 2.735297150749803,
  'жащ': 2.0722716091434545,
  'ащи': 1.2384223915119321,
  'щий': 2.378485230898969},
 'в': {},
 'биологии': {'био': 4.070465378911553,
  'иол': 4.8574082480242495,
  'оло': 1.7259106055839943,
  'лог': 2.6206542686

In [45]:
def binary_classification(dic):
    typos = []
    for k,v in dic.items():
        for key,val in v.items():
            if val > 4:
                typos.append((key, val)) 
                
            else:
                pass
    return typos
    

In [46]:
df["typo"] = df["dict_of_ngrams"].apply(lambda x: binary_classification(x))

In [49]:
df["classification"] = df["typo"].apply(lambda x: 1 if len(x) > 0 else 0)


In [51]:
df.head(3)

,text,processed,dict_of_ngrams,typo,classification
0,не обнаруживается различий в общем объеме серо...,"[не, обнаруживается, различий, в, общем, объем...","{'не': {}, 'обнаруживается': {'обн': 1.6718591...","[(чий, 4.768692534844364), (пац, 4.94659706913...",1
1,"интегумент ( от - покрывало , покров ) - терми...","[интегумент, от, покрывало, покров, термин, лу...","{'интегумент': {'инт': 2.326208970704359, 'нте...","[(био, 4.070465378911553), (иол, 4.85740824802...",1
2,"22 июня 2013 года решениме большинстав судей ,...","[июня, года, решениме, большинстав, судей, бик...","{'июня': {'июн': 1.3344986291917582, 'юня': 1....","[(бик, 6.311041097815426), (мек, 4.93024290800...",1


In [52]:
df.to_csv("df_with_typos.csv")

In [ ]:
def predict_candidates(word, k):
    # TODO: predict top k most similar words from russian word dictionary
    return candidates